# Lab 1

## Part I

Design and implement an application that generates a list of all communities, which include community
№1 members, by the community №1 identifier. The list should be ranked from the most popular community
to the least.
Working process:
1. Register in the social network VKontakte - vk.com
2. Create a stand-alone application https://vk.com/dev and get the application ID
3. Get a token for working with the VK API
4. Using any programming language, implement the function of displaying communities by popularity
among participants in the initial community No. 1 (the number of participants must be at least 2000)


In [1]:
import os
from modules.vkapi import VkApi
import pandas as pd

from dotenv import load_dotenv

load_dotenv()

True

C:\Users\hjklv\AppData\Local\pypoetry\Cache\virtualenvs\data-analysis-methods-99e_IxHd-py3.12\Lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


ИШИТР ТПУ - 73332691

In [2]:
members_with_subscriptions = (VkApi(os.getenv('VK_API_URL'), os.getenv('VK_ACCESS_TOKEN'))
       .get_members_with_subscriptions(73332691))
(len(members_with_subscriptions), members_with_subscriptions[0])

(3415,
 {'user_id': '27755',
  'response': {'response': {'count': 93,
    'items': [{'id': 70419166,
      'name': 'Сообщество Академгородка | Новосибирск',
      'screen_name': 'akadem54',
      'is_closed': 0,
      'type': 'page',
      'photo_50': 'https://sun139-2.userapi.com/s/v1/if1/mldZISguNb4iFM7uNp_OnXH1UCRSTynczQgWiOnKgT4yjRtGehPkmNp9HoHTbdhd5iTXwEyy.jpg?quality=96&crop=9,9,281,281&as=32x32,48x48,72x72,108x108,160x160,240x240&ava=1&cs=50x50',
      'photo_100': 'https://sun139-2.userapi.com/s/v1/if1/mldZISguNb4iFM7uNp_OnXH1UCRSTynczQgWiOnKgT4yjRtGehPkmNp9HoHTbdhd5iTXwEyy.jpg?quality=96&crop=9,9,281,281&as=32x32,48x48,72x72,108x108,160x160,240x240&ava=1&cs=100x100',
      'photo_200': 'https://sun139-2.userapi.com/s/v1/if1/mldZISguNb4iFM7uNp_OnXH1UCRSTynczQgWiOnKgT4yjRtGehPkmNp9HoHTbdhd5iTXwEyy.jpg?quality=96&crop=9,9,281,281&as=32x32,48x48,72x72,108x108,160x160,240x240&ava=1&cs=200x200'},
     {'id': 63109975,
      'name': 'Наш БАДМИНТОН',
      'screen_name': 'koposovaleks

In [3]:
import itertools

def map_subscriptions_to_id_name(subscription_item: (str, dict[str, str | int])):
    return {
        'user_id': subscription_item['user_id'],
        'id': subscription_item['id'], 
        'name': subscription_item.get('name'),
        'is_closed': subscription_item['is_closed'],
        'type': subscription_item['type'],
    }    

def add_user_id_to_items(user_id, items):
    def add_user_id(item):
        item['user_id'] = user_id
        return item
    return list(map(lambda x: add_user_id(x), items))

subscriptions = map(lambda x: add_user_id_to_items(x.get('user_id'), x.get('response').get('response').get('items')), filter(lambda x: x.get('response') is None or x.get('response').get('response'), members_with_subscriptions))
subscriptions = list(itertools.chain.from_iterable(subscriptions))
subscriptions = list(map(lambda x: map_subscriptions_to_id_name(x), subscriptions))

df = pd.DataFrame.from_dict(subscriptions)

In [4]:
df

,user_id,id,name,is_closed,type
0,27755,70419166,Сообщество Академгородка | Новосибирск,0,page
1,27755,63109975,Наш БАДМИНТОН,0,page
2,27755,212097236,Интернет-магазин «Товары для бадминтона»,0,page
3,27755,213729350,Миша Киселёв. Без Барьеров,0,page
4,27755,175231820,Мемный край,0,page
...,...,...,...,...,...
48087,868135195,751819357,None,False,profile
48088,868135195,674408425,None,False,profile
48089,874569846,201345296,Поиск команды | Mobile Legends,0,page
48090,874593288,196716535,Сейчас в Томске,0,page


C:\Users\hjklv\AppData\Local\pypoetry\Cache\virtualenvs\data-analysis-methods-99e_IxHd-py3.12\Lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [5]:
df['name'].value_counts().sort_values(ascending=False).head(3)

name
Подслушано в ТПУ           978
Регион-70 | Томск          477
AVTFamily | Актив ИШИТР    375
Name: count, dtype: int64

C:\Users\hjklv\AppData\Local\pypoetry\Cache\virtualenvs\data-analysis-methods-99e_IxHd-py3.12\Lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
